In [1]:
from sklearn.datasets import make_classification
import pandas as pd 
import numpy as np 
import random

In [3]:
X, y = make_classification(n_samples=400, n_features=4)
X = pd.DataFrame(X, columns=['f1', 'f2', 'f3', 'f4'])
y = pd.Series(y)

Нужно реализовать SVM с версией SGD 


In [6]:
random.seed(4)
for i in range(5):
    print(random.sample(range(11), 4))

[3, 4, 1, 6]
[7, 2, 1, 8]
[0, 6, 8, 4]
[0, 3, 8, 5]
[4, 2, 1, 10]


In [8]:
type(5.322213213213213123213892392382137913)

float

In [10]:
'''
 X_copy = X.copy().to_numpy()
        y_copy = y.replace(0, -1).to_numpy() # меняем метки таргетов {0, 1} -> {-1, 1}
        self.weights = np.ones(X_copy.shape[1])
        self.b = 1
''';

In [12]:
a = np.array([3,1,4,23,213,123,3,3241,123,43,12,12,54,65,234,321,6,8,5,3,7,32])

In [16]:
a[[4,2,3]]

array([213,   4,  23])

In [19]:
class MySVM():
    def __init__(self, n_iter=10, learning_rate=0.001, C=1, random_state=42, sgd_sample=None, weights=None, b=None):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.C = C
        self.sgd_sample = sgd_sample
        self.random_state = random_state
        self.__weights = weights
        self.__b = b # отступ гиперплоскости
        
    def __repr__(self):
        return f'MySVM class: n_iter={self.n_iter}, learning_rate={self.learning_rate}'
        
    def __calculate_loss(self, X, y):
        '''Cчитает LOSS'''
        scores = np.dot(X, self.__weights) + self.__b
        loss = np.dot(self.__weights, self.__weights) + self.C * np.sum(np.maximum(0, 1 - y * scores)) / len(y)
        return loss

    def __log_report(self, X, y, iteration):
        '''Выводит логи'''
        if iteration == 0:
            print(f'start | loss: {self.__calculate_loss(X, y).round(2)}')
        else:
            print(f'{iteration} | loss: {self.__calculate_loss(X, y).round(2)}')

    def __get_sgd_batch_size(self, row:int):
        '''Вернет размер батча для обучения SGD'''
        if type(self.sgd_sample) == int:
            return self.sgd_sample
        elif type(self.sgd_sample) == float:
            return max(1, int(row*self.sgd_sample))
        
    def fit(self, X:pd.DataFrame, y:pd.Series, verbose=False):
        random.seed(self.random_state)
        X_copy = X.copy().to_numpy()
        y_copy = y.replace(0, -1).to_numpy() # меняем метки таргетов {0, 1} -> {-1, 1}
        self.__weights = np.ones(X_copy.shape[1])
        self.__b = 1
       
        # Начнем цикл обчения
        for j in range(self.n_iter):   
            if self.sgd_sample is not None:                             # если у нас версия SGD, то для обучения формируем мини батчи  
                sample = self.__get_sgd_batch_size(len(y_copy))                 
                sample_idx = random.sample(range(X_copy.shape[0]), sample)    
                X_batc = X_copy[sample_idx]
                y_batc = y_copy[sample_idx]
            else:                                                       # иначе обучаемся на всех данных
                X_batc = X_copy
                y_batc = y_copy
                
            for i in range(X_batc.shape[0]): 
                if y_batc[i]*(np.dot(self.__weights, X_batc[i]) + self.__b) >= 1: 
                    grad_w = 2 * self.__weights
                    grad_b = 0 
                else:
                    grad_w = 2 * self.__weights - self.C * y_batc[i] * X_batc[i] 
                    grad_b = -self.C * y_batc[i]

                self.__weights -= self.learning_rate * grad_w
                self.__b -= self.learning_rate * grad_b

            if verbose and (j % 10 == 0 or j == self.n_iter-1):        # Loss считаем на всех данных, а не на мини-пакетах
                self.__log_report(X_copy, y_copy, j)

    def predict(self, X:pd.DataFrame):
        y = pd.Series(np.sign(np.dot(X, self.__weights) + self.__b))
        y = y.replace(-1, 0)
        return y.to_numpy().astype(int)

    def get_coef(self):
        return self.__weights, self.__b
        

In [22]:
model = MySVM(n_iter=100, learning_rate=0.05, C=0.5, sgd_sample=0.5)
model.fit(X,y, verbose=True)

start | loss: 0.4
10 | loss: 0.39
20 | loss: 0.4
30 | loss: 0.4
40 | loss: 0.39
50 | loss: 0.4
60 | loss: 0.39
70 | loss: 0.42
80 | loss: 0.39
90 | loss: 0.44
99 | loss: 0.39


In [24]:
model.predict(X.iloc[:5])

array([0, 1, 0, 0, 1])

In [26]:
y.iloc[:5].to_numpy()

array([0, 1, 0, 0, 1])

In [28]:
model.get_coef()

(array([-0.16114162,  0.01677588,  0.07101585,  0.20098249]),
 0.1499999999999993)

In [30]:
from sklearn.metrics import classification_report
print(classification_report(y, model.predict(X)))

              precision    recall  f1-score   support

           0       0.89      0.78      0.83       200
           1       0.80      0.90      0.85       200

    accuracy                           0.84       400
   macro avg       0.84      0.84      0.84       400
weighted avg       0.84      0.84      0.84       400

